In [1]:
# install.packages("ggplot2")
# install.packages('rms')
# install.packages("timereg")
# install.packages("svglite")
# install.packages("htmltools",version="0.5.7")
# install.packages("Hmisc")
library(ggplot2)
library(rms)
library(timereg)
# library(broom)
library(svglite)

Loading required package: Hmisc


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: survival



# Functions to find bounds and plot the data

In [2]:
find_bounds <- function(df, ...) {
    pre_data <- df[1,1]
    pre_y <- df[1,'yhat']
    bounds <- c()
    point <- c()
    for (i in 1:nrow(df)) {
        cur_y <- df[i,'yhat']
        cur_data <- df[i,1]
        if (pre_y > 1 && cur_y < 1){
            bounds <- c(bounds,c(cur_data))
            point <- c(point,'down')
        }
        if (pre_y <= 1 && cur_y > 1){
            bounds <- c(bounds,c(pre_data))
            point <- c(point,'up')
            
        }
        pre_data <- cur_data
        pre_y <- cur_y

    }
    return (bounds)
}
find_point <- function(df, ...) {
    pre_data <- df[1,1]
    pre_y <- df[1,'yhat']
    bounds <- c()
    point <- c()
    for (i in 1:nrow(df)) {
        cur_y <- df[i,'yhat']
        cur_data <- df[i,1]
        if (pre_y > 1 && cur_y < 1){
            bounds <- c(bounds,c(cur_data))
            point <- c(point,'down')
        }
        if (pre_y <= 1 && cur_y > 1){
            bounds <- c(bounds,c(pre_data))
            point <- c(point,'up')
            
        }
        pre_data <- cur_data
        pre_y <- cur_y

    }
    return (point)
}


In [3]:
save_pic_bounds <- function(HR,col,bounds,group,group_by,p,p_linear,is_first,col_for_pic){
    max_data = max(HR['data_col'])
    max_y = max(HR['upper'])
    pp<-ggplot()+
    geom_line(data=HR, aes(data_col,yhat),
            linetype="solid",size=1,alpha = 0.7,colour="#0070b9")+
    geom_ribbon(data=HR, 
              aes(data_col,ymin = lower, ymax = upper),
              alpha = 0.1,fill="#0070b9")+
    theme_classic()+
    geom_hline(yintercept=1, linetype=3,size=1)+
    theme(text = element_text(size = 22)) # set the font size of the plot
    
    if(group == 3)
        pp <- pp + labs(title = paste("Risk"),x=paste(col_for_pic), y="HR (95%CI)")
    else{
        strs = c("0"='Low-Albumin',"1"='Mid-Albumin',"2"='High-Albumin')
        pp <- pp + labs(title = paste("Risk ","(", strs[group],")",sep=""), x=paste(col_for_pic), y="HR (95%CI)")
        
    }
    
    # the vertical line
    i <-  0
    for (bound in bounds){
        pp <- pp + geom_vline(xintercept=bound, size=1, color='#d40e8c', linetype="dashed", alpha=0.5)
        if (i == 0){
            pp <- pp + annotate("text", x = bound, y = 0, label = paste('A:',round(bound,2)),size = 7)
        }else if(i == 1){
            pp <- pp + annotate("text", x = bound, y = 0.5, label = paste('B:',round(bound,2)),size = 7)
            
        }else{
            pp <- pp + annotate("text", x = bound, y = 1.3, label = paste('C:',round(bound,2)),size = 7)

        }
        i <- i + 1
    }
    
    # save the result
    filename = paste("result/statistics_r/res/time/r_pic_png/",group_by,"/",col,"_",group,".png",sep="")
    filename = paste("result/statistics_r/res/time/r_pic_svg/",group_by,"/",col,"_",group,".svg",sep="")
    filename = paste("result/statistics_r/res/time/r_pic_pdf/",group_by,"/",col,"_",group,".pdf",sep="")
    # get the folder path
    folder_path <- dirname(filename)
    
    if (!dir.exists(folder_path)) {
        # create the folder
        dir.create(folder_path, recursive = TRUE)
    }
    
    if (!file.exists(filename)) {
        file.create(filename)
    }
    pp <- pp + annotate("text", x = max_data*0.7, y = max_y*0.8, 
                    label = paste("p", ifelse(p < 0.001, "< 0.001", ifelse(p < 0.01, "< 0.01", paste("=", round(p, 2)))),
                                  "p(non-linear)", ifelse(p_linear < 0.001, "< 0.001", ifelse(p_linear < 0.01, "< 0.01", paste("=", round(p_linear, 2)))),
                                  sep=" "), size = 7)

    ggsave(
      filename = filename, # the filename (pdf/png/svg)
      width = 7,             # width
      height = 7,            # height
      units = "in",          # units
      dpi = 300              # dpi

    )
}

# Run the model and save the result for stage1 and stage2

In [4]:

vars <- c('water', 'protein', 'fat', 'carbohydrate', 'Calories', 'df', 
'k', 'na', 'mg', 'ca', 'p', 'fe', 'zn', 'cu', 'mn', 'se', 'retinol', 'vitaminA', 'carotene', 
'vitaminE', 'thiamine', 'riboflavin', 'nicotinic', 'ascorbic', 'dpi','dei')

vars_for_pic <- c('Date','Water (100g/d)','Protein (g/d)','Fat (10g/d)','Carbohydrate (g/d)','Calories (10kcal/d)','Df (g/d)',
                  'K (mg/d)','Na (100mg/d)',
            'Mg (10mg/d)','Ca (100mg/d)','P (mg/d)','Fe (mg/d)','Zn (mg/d)','Cu (mg/d)',
                  'Mn (mg/d)','Se (ug/d)','Retinol(ug/d)','VitaminA (100ugRAE/d)','Carotene (100ug/d)','VitaminE (ug/d)','Thiamine (mg/d)',
            'Riboflavin (mg/d)','Nicotinic (mgNE/d)','Ascorbic (mg/d)','DPI (0.1g/kg/d)','DEI (kcal/kg/d)')
stages <- c("0","1","2","3")
res <- c()
all <- 1

res_cox = c()
# for (group_by in c('Albumin','age','DBP','SBP','diab')){
for (group_by in c('Albumin')){
    if (group_by == 'diab'){
        stages <- c("0","1","2")
    }
    for (stage in stages){
        col_index <- 1
        for (col in vars){
            col_index <- col_index + 1
            
            filename <- paste('./result/statistics_r/group_data_time_new/group_by_',group_by,'_',col,'_',stage,'.csv',sep="")
            # print(filename)
            data <- read.csv(filename)
            # package the data
            dd <- datadist(data) #set the data environment for the code
            options(datadist='dd')

            
            
            # fit the model
            if (group_by == 'diab'){

                fit<- cph(Surv(live_time,death) ~ rcs(data_col,4)+Cl+CO2CP+Urea+Na+Scr+P+Albumin+hsCRP+Glucose+Weight+SBP+DBP+gfr+K+bmi+WBC+age+height,data=data)  # 节点数设为4
                
            }else{
                fit<- cph(Surv(live_time,death) ~ rcs(data_col,4)+diab+Cl+CO2CP+Urea+Na+Scr+P+hsCRP+Glucose+Weight+SBP+DBP+gfr+K+bmi+WBC+age+height,data=data)  # 节点数设为4
                res_coxph<- coxph(Surv(live_time,death) ~ data_col+Cl+CO2CP+Urea+Na+Scr+P+Albumin+hsCRP+Glucose+Weight+SBP+DBP+gfr+K+bmi+WBC+age+height,data=data)  # 节点数设为4
                # risk_score <- predict(res_coxph, newdata=data, type='risk')
                # roc_curve <-timeROC(Surv(live_time, death)~risk_score, data=data, times=c(1))
                # plot(roc_curve)
                
            }
            line_1 = as.matrix(summary(res_coxph)$coefficients)[1,]
            confidence_interval = exp(confint(res_coxph)[1, ])
            # print(confidence_interval[1])
            line = c(col,stage,line_1[1],line_1[2],line_1[3],line_1[4],line_1[5],confidence_interval[1],confidence_interval[2])
            res_cox = c(res_cox,line)
            res_cox_all  <- matrix(res_cox, ncol = 9,byrow = TRUE) # 行的顺序不变，不然会乱序
            colnames(res_cox_all) <- c("col","stage","c","c1","c2","z","p",'951','952')
            
    
            # non-linear test
            # P<0.05 denotes that the non-linear effect is significant
            anova_res = anova(fit)
            # print(anova_res)
            # print(anova_res)
            p <- anova_res[1,3]
            p_linear <- anova_res[2,3]
            
            HR<-Predict(fit, data_col,fun=exp,ref.zero = TRUE)
            bounds = find_bounds(HR)
            point = find_point(HR)
            save_pic_bounds(HR,col,bounds,stage,group_by,p,p_linear,is_first,vars_for_pic[col_index])
            b1 = bounds[1]

            while(length(bounds)<3){ # there are 1-3 lines, if less than 3, fill it up to 3
                bounds = c(bounds,c(-1))
            }
            while(length(point)<3){ # there are 1-3 lines, if less than 3, fill it up to 3
                point = c(point,c(-1))
            }
            row <-c(col,stage,round(bounds[1],2),round(bounds[2],2),round(bounds[3],2),round(p,4),round(p_linear,4),point[1],point[2],point[3])
            res <- c(res,row)
            all  <- matrix(res, ncol = 10,byrow = TRUE) # keep the order of the rows, otherwise it will be disordered
            colnames(all) <- c("diet","stage","round1","round2","round3","p","p_nonlinear","d1","d2","d3")
        }
    }
    write.csv(all,file=paste("result/statistics_r/res/time/res_",group_by,".csv",sep=''))
    write.csv(res_cox_all,file=paste("result/statistics_r/res/time/res_cox_",group_by,".csv",sep=''))
    print(paste(" the result of stage1 is saved at result/statistics_r/res/time/res_cox_",group_by,".csv",sep=''))
    print(paste(" the result of stage2 is saved at result/statistics_r/res/time/res_",group_by,".csv",sep=''))
}


Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”


[1] " the result of stage1 is saved at result/statistics_r/res/time/res_cox_Albumin.csv"
[1] " the result of stage2 is saved at result/statistics_r/res/time/res_Albumin.csv"
